In [ ]:
import torch

import numpy as np
import pandas as pd

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast

from src.data_utils import clean_string, TextDataset, collate_fn
from src.lstm_model import lstm

e:\DS\HM\yandex\tweets\prac_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'src.lstm_model'

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Используем {device}')

Используем cpu


### Этап 1. Сбор и подготовка данных
#### Загрузка, очистка, токенизация датасета

#### не забыть вернуть полный датасет

In [2]:
with open('data/tweets.txt', 'r', encoding='utf-8') as f:
    texts = f.readlines()

df = pd.DataFrame({'text': texts[:1000]})

In [3]:
df['clean_text'] = [clean_string(text) for text in tqdm(df['text'])]
df = df[df['clean_text'].str.len() > 0].reset_index(drop=True)

100%|██████████| 1000/1000 [00:00<00:00, 14807.78it/s]


In [4]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

tokenized_lengths = []
for text in df['clean_text'].head(10000):
    tokens = tokenizer.tokenize(text)
    tokenized_lengths.append(len(tokens))

MAX_LENGTH = int(np.percentile(tokenized_lengths, 95))

encoded = tokenizer(
    df['clean_text'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=MAX_LENGTH,
    return_tensors='pt'
    )

#### Разделение на train, val, test

In [5]:
train_ids, temp_ids, train_mask, temp_mask = train_test_split(
    encoded['input_ids'], 
    encoded['attention_mask'], 
    test_size=0.2, 
    random_state=42
)

val_ids, test_ids, val_mask, test_mask = train_test_split(
    temp_ids, 
    temp_mask, 
    test_size=0.5, 
    random_state=42
)

#### Создание dataset и dataloader

In [6]:
BATCH_SIZE = 256

train_dataset = TextDataset(train_ids, train_mask)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers=0)

val_dataset = TextDataset(val_ids, val_mask)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=0)

test_dataset = TextDataset(test_ids, test_mask)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=0)

print(f"Train: {len(train_dataset)}")
print(f"Val: {len(val_dataset)}")
print(f"Test: {len(test_dataset)}")

Train: 796
Val: 99
Test: 100


### Этап 2. Реализация рекуррентной сети

In [10]:
model = LSTM(vocab_size=encoded.vocab_size, embedding_dim=128,
            hidden_size=128, num_layers=2,
            use_bidirectional=True)
model.to(device)

NameError: name 'LSTM' is not defined